In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split 

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [55]:
charging_set = pd.read_csv("data/charging_sessions.csv")

In [41]:
# get overview of data
print("columns:")
print(charging_set.columns)
print("-------------------------------------------------------------------------")
print("info():")
print(charging_set.info())
print("-------------------------------------------------------------------------")
print("describe():")
print(charging_set.describe())

columns:
Index(['Unnamed: 0', 'id', 'connectionTime', 'disconnectTime',
       'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID',
       'stationID', 'timezone', 'userID', 'userInputs'],
      dtype='object')
-------------------------------------------------------------------------
info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone    

In [42]:
charging_set.head(10)

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."
5,5,5e23b149f9af8b5fe4b973d4,2020-01-02 14:00:13+00:00,2020-01-02 23:00:41+00:00,2020-01-02 20:13:21+00:00,13.370,1_1_194_821_2020-01-02 14:00:05.241573,1,AG-1F13,1-1-194-821,America/Los_Angeles,3519.0,"[{'WhPerMile': 600, 'kWhRequested': 24.0, 'mil..."
6,6,5e23b149f9af8b5fe4b973d5,2020-01-02 14:09:14+00:00,2020-01-03 01:57:58+00:00,2020-01-03 00:59:21+00:00,43.477,1_1_178_817_2020-01-02 14:09:14.451827,1,AG-1F09,1-1-178-817,America/Los_Angeles,933.0,"[{'WhPerMile': 385, 'kWhRequested': 65.45, 'mi..."
7,7,5e23b149f9af8b5fe4b973d6,2020-01-02 14:17:32+00:00,2020-01-03 00:44:54+00:00,2020-01-02 20:22:40+00:00,32.441,1_1_191_804_2020-01-02 14:17:32.329535,1,AG-4F34,1-1-191-804,America/Los_Angeles,3737.0,"[{'WhPerMile': 300, 'kWhRequested': 45.0, 'mil..."
8,8,5e23b149f9af8b5fe4b973d7,2020-01-02 14:25:38+00:00,2020-01-02 19:36:05+00:00,2020-01-02 18:38:39+00:00,13.264,1_1_194_826_2020-01-02 14:25:37.578692,1,AG-1F11,1-1-194-826,America/Los_Angeles,419.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil..."
9,9,5e23b149f9af8b5fe4b973d8,2020-01-02 14:27:40+00:00,2020-01-02 20:01:48+00:00,2020-01-02 19:17:46+00:00,13.285,1_1_178_823_2020-01-02 14:27:39.582337,1,AG-1F08,1-1-178-823,America/Los_Angeles,651.0,"[{'WhPerMile': 400, 'kWhRequested': 20.0, 'mil..."


# TODO

- check validity of columns
- check all columns for Na-values e.g. NaN, NaT
- check all columns for wrong data types
- check all columns for unidentified characters e.g. "$"

Notes:
- doneChargingtime has NaT values --> possible interpretation: EV is discconnected from charging hub before reaching 100% state of charge
- what to do with IDs
- userID has NaN values --> not all users are registered

# Data Cleanup

## Step 1: check validity of columns

- remove unnecessary "unnamed" column
- "sesionID" = combination of "id"
- check ranges of columns -->

In [56]:
# remove irrelevant columns
charging_set.drop('Unnamed: 0', axis=1, inplace=True)
charging_set.drop('sessionID', axis=1, inplace=True) # does not bring relevant information for our analysis
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."


In [ ]:
# split userInputs into own columns


In [59]:
# check for duplicates
duplicate_counter = charging_set.duplicated().sum()
duplicates = charging_set[charging_set.duplicated()]
print("sum of duplicates:", duplicate_counter)
charging_set.drop_duplicates()


sum of duplicates: 1413


,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."
...,...,...,...,...,...,...,...,...,...,...,...
65032,5d2fbdd3f9af8b4d0dd0d54a,2019-07-01 21:49:12+00:00,2019-07-02 01:37:18+00:00,2019-07-01 23:14:33+00:00,8.399,1,AG-3F18,1-1-179-798,America/Los_Angeles,1346.0,"[{'WhPerMile': 400, 'kWhRequested': 32.0, 'mil..."
65033,5d2fbdd3f9af8b4d0dd0d54b,2019-07-01 21:58:45+00:00,2019-07-02 00:39:48+00:00,2019-07-02 00:40:21+00:00,16.864,1,AG-3F20,1-1-179-794,America/Los_Angeles,364.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil..."
65034,5d2fbdd3f9af8b4d0dd0d54c,2019-07-01 22:02:21+00:00,2019-07-02 00:58:50+00:00,2019-07-02 00:59:23+00:00,18.335,1,AG-4F47,1-1-191-807,America/Los_Angeles,2050.0,"[{'WhPerMile': 333, 'kWhRequested': 29.97, 'mi..."
65035,5d2fbdd3f9af8b4d0dd0d54d,2019-07-01 22:23:44+00:00,2019-07-02 02:03:36+00:00,2019-07-02 02:04:06+00:00,22.815,1,AG-3F31,1-1-179-781,America/Los_Angeles,1626.0,"[{'WhPerMile': 200, 'kWhRequested': 34.0, 'mil..."


In [60]:
# check for none-values
nan_counts = charging_set.isna().sum()
nan_counts_sorted = nan_counts.sort_values(ascending=False)

for column, count in nan_counts_sorted.items():
    print(f"Feature: {column}, Count of NaN-Values: {count}")

Feature: userID, Count of NaN-Values: 17263
Feature: userInputs, Count of NaN-Values: 17263
Feature: doneChargingTime, Count of NaN-Values: 4088
Feature: disconnectTime, Count of NaN-Values: 0
Feature: connectionTime, Count of NaN-Values: 0
Feature: id, Count of NaN-Values: 0
Feature: kWhDelivered, Count of NaN-Values: 0
Feature: spaceID, Count of NaN-Values: 0
Feature: siteID, Count of NaN-Values: 0
Feature: timezone, Count of NaN-Values: 0
Feature: stationID, Count of NaN-Values: 0


In [54]:
### check data types of each column
# Find the unique types
id_types = charging_set["id"].apply(type).unique()
connectionTime_types = charging_set["connectionTime"].apply(type).unique()
disconnectTime_types = charging_set["disconnectTime"].apply(type).unique()
doneChargingTime_types = charging_set["doneChargingTime"].apply(type).unique()
kWhDelivered_types = charging_set["kWhDelivered"].apply(type).unique()
siteID_types = charging_set["siteID"].apply(type).unique()
spaceID_types = charging_set["spaceID"].apply(type).unique()
stationID_types = charging_set["stationID"].apply(type).unique()
timezone_types = charging_set["timezone"].apply(type).unique()
userID_types = charging_set["userID"].apply(type).unique()
userInputs_types = charging_set["userInputs"].apply(type).unique()

# Print the unique types
print("id types:", id_types)
print("connectionTime types:", connectionTime_types)
print("disconnectTime types:", disconnectTime_types)
print("doneChargingTime types:", doneChargingTime_types)
print("kWhDelivered types:", kWhDelivered_types)
print("siteID types:", siteID_types)
print("spaceID types:", spaceID_types)
print("stationID types:", stationID_types)
print("timezone types:", timezone_types)
print("userID types:", userID_types)
print("userInputs types:", userInputs_types)

id types: [<class 'str'>]
connectionTime types: [<class 'str'>]
disconnectTime types: [<class 'str'>]
doneChargingTime types: [<class 'str'> <class 'float'>]
kWhDelivered types: [<class 'float'>]
siteID types: [<class 'int'>]
spaceID types: [<class 'str'>]
stationID types: [<class 'str'>]
timezone types: [<class 'str'>]
userID types: [<class 'float'>]
userInputs types: [<class 'str'> <class 'float'>]


In [51]:
##### check ranges of column values
print("id data range:", np.sort(charging_set["id"].unique()))
print("connectionTime data range:", np.sort(charging_set["connectionTime"].unique()))
print("disconnectTime data range:", np.sort(charging_set["disconnectTime"].unique()))
print("doneChargingTime data range:", np.sort(charging_set["doneChargingTime"].unique()))
print("kWhDelivered data range:", np.sort(charging_set["kWhDelivered"].unique()))
print("sessionID data range:", np.sort(charging_set["sessionID"].unique()))
print("siteID data range:", np.sort(charging_set["siteID"].unique()))
print("spaceID data range:", np.sort(charging_set["spaceID"].unique()))
print("stationID data range:", np.sort(charging_set["stationID"].unique()))
print("timezone data range:", np.sort(charging_set["timezone"].unique()))
print("userID data range:", np.sort(charging_set["userID"].unique()))
print("userInputs data range:", np.sort(charging_set["userInputs"].unique()))
print("connectionTime_date data range:", np.sort(charging_set["connectionTime_date"].unique()))
print("connectionTime_time data range:", np.sort(charging_set["connectionTime_time"].unique()))
print("disconnectTime_date data range:", np.sort(charging_set["disconnectTime_date"].unique()))
print("disconnectTime_time data range:", np.sort(charging_set["disconnectTime_time"].unique()))
print("doneChargingTime_date data range:", np.sort(charging_set["doneChargingTime_date"].unique()))
print("doneChargingTime_time data range:", np.sort(charging_set["doneChargingTime_time"].unique()))

id data range: ['5bc90cb9f9af8b0d7fe77cd2' '5bc90cb9f9af8b0d7fe77cd3'
 '5bc90cb9f9af8b0d7fe77cd4' ... '6155053bf9af8b76960e16cf'
 '6155053bf9af8b76960e16d0' '6155053bf9af8b76960e16d1']
connectionTime data range: ['2018-04-25 11:08:04+00:00' '2018-04-25 13:45:10+00:00'
 '2018-04-25 13:45:50+00:00' ... '2021-09-14 01:08:16+00:00'
 '2021-09-14 01:52:37+00:00' '2021-09-14 05:43:39+00:00']
disconnectTime data range: ['2018-04-25 13:20:10+00:00' '2018-04-25 18:15:46+00:00'
 '2018-04-25 18:27:51+00:00' ... '2021-09-14 03:05:10+00:00'
 '2021-09-14 03:17:55+00:00' '2021-09-14 14:46:28+00:00']


TypeError: '<' not supported between instances of 'float' and 'str'

Since the "timezone" column only has the value "America/Los Angeles", we can remove this column.

In [11]:
charging_set.drop('timezone', axis=1, inplace=True)
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."


id types: [<class 'str'>]
connectionTime types: [<class 'str'>]
disconnectTime types: [<class 'str'>]
doneChargingTime types: [<class 'str'> <class 'float'>]
kWhDelivered types: [<class 'float'>]
siteID types: [<class 'int'>]
spaceID types: [<class 'str'>]
stationID types: [<class 'str'>]
userID types: [<class 'float'>]
userInputs types: [<class 'str'> <class 'float'>]


Next step: analyze "connectionTime", "disconnectTime" and "doneChargingTime", since these columns should hold time-datatypes and not strings nor floats (except for NaN-values since these have the type float).

# Time Data Preparation/Feature Engineering (draft)

For better data understanding and to improve data accessibility and interpretability, it is important to convert the current time data format into a human-readable time datatype. Currently the time related columns ("connectionTime", "disconnectionTime", "doneChargingTime") are formatted in the UTC format. To prepare the data for further interpretation we need to gather the relevant information. In this case the important information is the charging duration.

To get the duration we divide each of the time related columns into separate date and datetime columns. With these columns, the difference between connecting the EV with the station ("connectionTime") and disconnecting the EV with the station ("disconnectTime") can be calculated and stored in a separate column ("chargingDuration").


In [14]:
### split connectionTime in separate day and time columns
# Convert the column to datetime format
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])

# Create a new column for the date
charging_set['connectionTime_date'] = charging_set['connectionTime'].dt.date

# Create a new column for the time of the day
charging_set['connectionTime_time'] = charging_set['connectionTime'].dt.time


### split disconnectTime in separate day and time columns
# Convert the column to datetime format
charging_set['disconnectTime'] = pd.to_datetime(charging_set['disconnectTime'])

# Create a new column for the date
charging_set['disconnectTime_date'] = charging_set['disconnectTime'].dt.date

# Create a new column for the time of the day
charging_set['disconnectTime_time'] = charging_set['disconnectTime'].dt.time


### split doneChargingTime in separate day and time columns
# Convert the column to datetime format
charging_set['doneChargingTime'] = pd.to_datetime(charging_set['doneChargingTime'])

# Create a new column for the date
charging_set['doneChargingTime_date'] = charging_set['doneChargingTime'].dt.date

# Create a new column for the time of the day
charging_set['doneChargingTime_time'] = charging_set['doneChargingTime'].dt.time

In [15]:
### convert date columns into separate year, month and day columns

# Das muss rein, weil sonst kommt ein Fehler
charging_set['connectionTime_date'] = pd.to_datetime(charging_set['connectionTime_date'])
charging_set['disconnectTime_date'] = pd.to_datetime(charging_set['disconnectTime_date'])
charging_set['doneChargingTime_date'] = pd.to_datetime(charging_set['doneChargingTime_date'])

charging_set['connectionTime_year'] = charging_set['connectionTime_date'].dt.year
charging_set['connectionTime_month'] = charging_set['connectionTime_date'].dt.month
charging_set['connectionTime_day'] = charging_set['connectionTime_date'].dt.day

charging_set['disconnectTime_year'] = charging_set['disconnectTime_date'].dt.year
charging_set['disconnectTime_month'] = charging_set['disconnectTime_date'].dt.month
charging_set['disconnectTime_day'] = charging_set['disconnectTime_date'].dt.day

charging_set['doneChargingTime_year'] = charging_set['doneChargingTime_date'].dt.year
charging_set['doneChargingTime_month'] = charging_set['doneChargingTime_date'].dt.month
charging_set['doneChargingTime_day'] = charging_set['doneChargingTime_date'].dt.day

In [16]:
def calculate_hours():
    # Berechne die Ladezeit als Differenz zwischen disconnectTime und connectionTime
    charging_set["chargingDurationTemp"] = charging_set["disconnectTime"] - charging_set["connectionTime"]
    
    # Konvertiere die Differenz in Stunden
    charging_set['chargingDuration'] = charging_set["chargingDurationTemp"].apply(
        lambda x: x.total_seconds() / 3600
    )
    
    # Entferne die temporäre Spalte
    charging_set.drop(columns=["chargingDurationTemp"], inplace=True)
    
    return charging_set

# Aufruf der Funktion
charging_set = calculate_hours()

In [17]:
# clean "connectionTime", "disconnectTime", "doneChargingTime" from dataframe
charging_set = charging_set.drop(columns=["connectionTime", "disconnectTime", "doneChargingTime"])

In [18]:
### check column "doneChargingTime" for wrong datatype
# hypothesis: all float values in this column are nan-values
# Find the indices of float values in a column that should contain only strings
# float_indices = [idx for idx, value in charging_set['doneChargingTime'].items() if isinstance(value, float) and not isinstance(value, str)]

# Print the result
# if float_indices:
#     print("Indices of float values in column 'doneChargingTime' (all others should be strings):", float_indices)
# else:
#     print("No float values found in the column.")

In [19]:
# Get indices where 'column_name' has NaN values
# nan_indices = charging_set[charging_set['doneChargingTime'].isna()].index.tolist()

# Print the indices of NaN values
#  print(nan_indices)

sum of duplicates: 1413
                             id  kWhDelivered  siteID  spaceID    stationID  \
65037  5d2fbdd3f9af8b4d0dd0d54f         2.463       1  AG-1F02  1-1-193-827   
65038  5d310f54f9af8b52adda3e20        12.891       1  AG-1F14  1-1-194-818   
65039  5d310f54f9af8b52adda3e21        13.734       1  AG-1F10  1-1-178-828   
65040  5d310f54f9af8b52adda3e22        10.619       1  AG-1F08  1-1-178-823   
65041  5d310f54f9af8b52adda3e23        19.079       1  AG-3F25  1-1-179-799   
...                         ...           ...     ...      ...          ...   
66445  5d574ad2f9af8b4c10c03652        28.787       1  AG-3F27  1-1-179-809   
66446  5d574ad2f9af8b4c10c03653         7.787       1  AG-3F30  1-1-179-810   
66447  5d574ad2f9af8b4c10c03654        11.274       1  AG-4F51  1-1-191-795   
66448  5d574ad2f9af8b4c10c03655        11.589       1  AG-4F43  1-1-191-778   
66449  5d574ad2f9af8b4c10c03656         0.897       1  AG-1F09  1-1-178-817   

       userID              

,id,kWhDelivered,siteID,spaceID,stationID,userID,userInputs,connectionTime_date,connectionTime_time,disconnectTime_date,...,connectionTime_year,connectionTime_month,connectionTime_day,disconnectTime_year,disconnectTime_month,disconnectTime_day,doneChargingTime_year,doneChargingTime_month,doneChargingTime_day,chargingDuration
0,5e23b149f9af8b5fe4b973cf,25.016,1,AG-3F30,1-1-179-810,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...",2020-01-02,13:08:54,2020-01-02,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,6.039167
1,5e23b149f9af8b5fe4b973d0,33.097,1,AG-1F01,1-1-193-825,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...",2020-01-02,13:36:50,2020-01-02,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,9.025278
2,5e23b149f9af8b5fe4b973d1,6.521,1,AG-1F03,1-1-193-829,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:56:35,2020-01-03,...,2020,1,2,2020,1,3,2020.0,1.0,2.0,10.713056
3,5e23b149f9af8b5fe4b973d2,2.355,1,AG-1F04,1-1-193-820,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:59:58,2020-01-02,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,2.644722
4,5e23b149f9af8b5fe4b973d3,13.375,1,AG-1F06,1-1-193-819,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...",2020-01-02,14:00:01,2020-01-02,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,8.144167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65032,5d2fbdd3f9af8b4d0dd0d54a,8.399,1,AG-3F18,1-1-179-798,1346.0,"[{'WhPerMile': 400, 'kWhRequested': 32.0, 'mil...",2019-07-01,21:49:12,2019-07-02,...,2019,7,1,2019,7,2,2019.0,7.0,1.0,3.801667
65033,5d2fbdd3f9af8b4d0dd0d54b,16.864,1,AG-3F20,1-1-179-794,364.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil...",2019-07-01,21:58:45,2019-07-02,...,2019,7,1,2019,7,2,2019.0,7.0,2.0,2.684167
65034,5d2fbdd3f9af8b4d0dd0d54c,18.335,1,AG-4F47,1-1-191-807,2050.0,"[{'WhPerMile': 333, 'kWhRequested': 29.97, 'mi...",2019-07-01,22:02:21,2019-07-02,...,2019,7,1,2019,7,2,2019.0,7.0,2.0,2.941389
65035,5d2fbdd3f9af8b4d0dd0d54d,22.815,1,AG-3F31,1-1-179-781,1626.0,"[{'WhPerMile': 200, 'kWhRequested': 34.0, 'mil...",2019-07-01,22:23:44,2019-07-02,...,2019,7,1,2019,7,2,2019.0,7.0,2.0,3.664444


In [22]:
charging_set.columns

Index(['id', 'kWhDelivered', 'siteID', 'spaceID', 'stationID', 'userID',
       'userInputs', 'connectionTime_date', 'connectionTime_time',
       'disconnectTime_date', 'disconnectTime_time', 'doneChargingTime_date',
       'doneChargingTime_time', 'connectionTime_year', 'connectionTime_month',
       'connectionTime_day', 'disconnectTime_year', 'disconnectTime_month',
       'disconnectTime_day', 'doneChargingTime_year', 'doneChargingTime_month',
       'doneChargingTime_day', 'chargingDuration'],
      dtype='object')

#### Anschauen User Inputs

In [23]:
charging_set["userInputs"].unique

<bound method Series.unique of 0        [{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...
1        [{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...
2        [{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...
3        [{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...
4        [{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...
                               ...                        
66445    [{'WhPerMile': 240, 'kWhRequested': 31.2, 'mil...
66446    [{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil...
66447    [{'WhPerMile': 333, 'kWhRequested': 19.98, 'mi...
66448    [{'WhPerMile': 275, 'kWhRequested': 22.0, 'mil...
66449                                                  NaN
Name: userInputs, Length: 66450, dtype: object>

Idee: 
- entferne temporär erstmal alle nan Werte um zum checken, ob die Funktion überhaupt funktioniert
- erstelle dafür eine temp datei
- danach sollte man sich mit dem auffüllen / entfernen von NaNs beschäftigen

In [24]:
charging_set

,id,kWhDelivered,siteID,spaceID,stationID,userID,userInputs,connectionTime_date,connectionTime_time,disconnectTime_date,...,connectionTime_year,connectionTime_month,connectionTime_day,disconnectTime_year,disconnectTime_month,disconnectTime_day,doneChargingTime_year,doneChargingTime_month,doneChargingTime_day,chargingDuration
0,5e23b149f9af8b5fe4b973cf,25.016,1,AG-3F30,1-1-179-810,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...",2020-01-02,13:08:54,2020-01-02,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,6.039167
1,5e23b149f9af8b5fe4b973d0,33.097,1,AG-1F01,1-1-193-825,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...",2020-01-02,13:36:50,2020-01-02,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,9.025278
2,5e23b149f9af8b5fe4b973d1,6.521,1,AG-1F03,1-1-193-829,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:56:35,2020-01-03,...,2020,1,2,2020,1,3,2020.0,1.0,2.0,10.713056
3,5e23b149f9af8b5fe4b973d2,2.355,1,AG-1F04,1-1-193-820,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:59:58,2020-01-02,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,2.644722
4,5e23b149f9af8b5fe4b973d3,13.375,1,AG-1F06,1-1-193-819,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...",2020-01-02,14:00:01,2020-01-02,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,8.144167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,5d574ad2f9af8b4c10c03652,28.787,1,AG-3F27,1-1-179-809,393.0,"[{'WhPerMile': 240, 'kWhRequested': 31.2, 'mil...",2019-07-31,18:08:04,2019-07-31,...,2019,7,31,2019,7,31,2019.0,7.0,31.0,5.353889
66446,5d574ad2f9af8b4c10c03653,7.787,1,AG-3F30,1-1-179-810,220.0,"[{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil...",2019-07-31,18:40:41,2019-08-01,...,2019,7,31,2019,8,1,2019.0,7.0,31.0,6.316944
66447,5d574ad2f9af8b4c10c03654,11.274,1,AG-4F51,1-1-191-795,1974.0,"[{'WhPerMile': 333, 'kWhRequested': 19.98, 'mi...",2019-07-31,19:04:40,2019-07-31,...,2019,7,31,2019,7,31,2019.0,7.0,31.0,3.661667
66448,5d574ad2f9af8b4c10c03655,11.589,1,AG-4F43,1-1-191-778,942.0,"[{'WhPerMile': 275, 'kWhRequested': 22.0, 'mil...",2019-07-31,19:19:47,2019-08-01,...,2019,7,31,2019,8,1,2019.0,7.0,31.0,5.251111
